In [ ]:
import pandas as pd
import random

# Cargar los datos
docentes = pd.read_csv("docentes.csv")
materias = pd.read_csv("materias.csv")
paralelos = pd.read_csv("paralelos.csv")
horarios = pd.read_csv("horarios.csv")

### 🧹 1. Unificación de docentes de Inglés

Este bloque estandariza los docentes que dictan Inglés a partir de 7mo EGB. Se eliminan los registros de las docentes "ELIZABETH PUGA" y "SIMONE AYALA", y se reemplaza a "DAISY HERRERA" por el identificador genérico `"INGLES ESPECIAL"`.

Este paso facilita la asignación conjunta de horarios en niveles superiores donde el docente de inglés es compartido entre paralelos.


In [ ]:
#Unificar docentes de inglés a partir de 7mo EGB
docentes=docentes.drop(docentes[docentes['Nombres']=='ELIZABETH PUGA'].index)
docentes=docentes.drop(docentes[docentes['Nombres']=='SIMONE AYALA'].index)
docentes["Nombres"] = docentes["Nombres"].replace("DAISY HERRERA", "INGLES ESPECIAL")



### 🏃 2. Unificación de docentes de Educación Física

De forma similar, se agrupan los docentes de Educación Física a partir de 3ro EGB. Se eliminan "CARLOS MORALES" e "IVÁN MALDONADO" y se reemplaza a "NANCY LINCANGO" por `"EF ESPECIAL"`.

Esto permite manejar la materia como una asignación especial compartida, lo cual es fundamental para las restricciones del modelo de horarios.


In [ ]:
#Unificar docentes de Educación Física a partir de 3ro EGB
docentes=docentes.drop(docentes[docentes['Nombres']=='CARLOS MORALES'].index)
docentes=docentes.drop(docentes[docentes['Nombres']=='IVÁN MALDONADO'].index)
docentes["Nombres"] = docentes["Nombres"].replace("NANCY LINCANGO", "EF ESPECIAL")

docentes.reset_index()

### 🔄 3. Normalización y expansión de códigos en `docentes`

Se crea una copia del DataFrame original y se limpia la columna `Codigo` eliminando los caracteres `{}`, `'`, que provienen del formato original del set de datos.

Luego, se separan los múltiples códigos almacenados en una sola celda (separados por comas) y se expande cada código a una fila distinta mediante `explode()`.

Esto permite trabajar con asignaciones individuales por fila, una estructura más adecuada para algoritmos evolutivos que requieren el análisis granular de clases por docente.


In [ ]:
df=docentes.copy()
# Limpiar la columna 'Codigo': eliminar llaves y comillas simples
df["Codigo"] = df["Codigo"].str.replace(r"[{}']", "", regex=True)

# Separar los códigos por coma y expandir en filas individuales
docentes = df.assign(Codigo=df["Codigo"].str.split(", ")).explode("Codigo")

### 🧠 4. Construcción del diccionario de asignación docente (`dic_profesores`)

Este bloque itera sobre cada fila del DataFrame `docentes` y construye un diccionario llamado `dic_profesores` con el siguiente mapeo:

```python
{ código_materia → nombre_docente }


In [ ]:
df=docentes.copy()
# Carga del archivo CSV
df = docentes

# Identifica las columnas
col_codigo = 'Codigo'
col_profesor = 'Nombres'

# Diccionario resultado
dic_profesores = {}

for _, row in df.iterrows():
    raw = row[col_codigo]
    if pd.isna(raw):
        continue

    # Convertimos a cadena y eliminamos espacios exteriores
    s = str(raw).strip()

    # Si empieza por '{' o '[' acabamos quitando delimitadores
    if s.startswith('{') and s.endswith('}'):
        # Quitamos las llaves
        s = s[1:-1]
    elif s.startswith('[') and s.endswith(']'):
        # Quitamos los corchetes
        s = s[1:-1]

    # Ahora s es algo como "'LEC_1EGB', 'LEC_3EGB', 'LEC_2EGB'"
    # Partimos por comas y limpiamos comillas
    codes = []
    for part in s.split(','):
        part = part.strip().strip("'\"")  # quita comillas simples o dobles
        if part:
            codes.append(part)

    # Asociamos cada código con el profesor
    for code in codes:
        dic_profesores[code] = row[col_profesor]

# Ejemplo de uso:
for ejemplo in ['LEN_3EGB', 'PEN_10EGB', 'EST_10EGB']:
    prof = dic_profesores.get(ejemplo, 'No encontrado')
    print(f"Profesor de {ejemplo}: {prof}")

Profesor de LEN_3EGB: SOFÍA YÉPEZ
Profesor de PEN_10EGB: MIREYA BRITO
Profesor de EST_10EGB: JONATHAN CASTRO


In [ ]:
materias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Materias  299 non-null    object
 1   Nivel     299 non-null    object
 2   Código    299 non-null    object
 3   Horas     299 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 9.5+ KB


### ⚠️ 5. Evaluación de restricciones duras (`evaluar_restricciones_duras`)

Esta función penaliza cromosomas que incumplen las restricciones duras del problema de horarios. Se basa en la estructura `(nivel, aula, día, hora, materia, docente)` y retorna una suma de penalizaciones acumuladas.

#### 📌 Subrestricciones implementadas:

1. **Sesiones no continuas innecesarias:**
   - Si una materia tiene más de una hora semanal y aparece separada por más de 2 bloques en un mismo día, se penaliza (0.1 puntos por interrupción).
   - Excluye materias que aparecen por primera vez o cambian de día.

2. **Más de 3 horas continuas de una misma materia:**
   - Si una materia aparece en más de 3 bloques seguidos dentro del mismo día, se penaliza (+2 puntos).
   - Excepciones específicas para `MAT_1EGB`, `LEN_1EGB`, y `MAT_2EGB`.

3. **Cruces docentes en el mismo horario:**
   - Si un docente aparece asignado a más de un paralelo en el mismo bloque horario (día y hora), se penaliza con +5 puntos por cada conflicto adicional.
   - Se aplican excepciones para:
     - `"EF ESPECIAL"` y `"INGLES ESPECIAL"`
     - `"JONATHAN CASTRO"` y `"JANETH MELO"` en `3BGU`
     - `"CIU_1BGU"` en `1BGU` y `"CIU_2BGU"` en `2BGU`

Esta evaluación permite al algoritmo evolutivo evitar asignaciones inviables y guiar la generación de soluciones factibles para el modelo escolar.


In [ ]:
from collections import defaultdict

# Asumimos que este diccionario ya fue creado previamente
horas_por_materia = dict(zip(materias["Código"], materias["Horas"]))

def evaluar_restricciones_duras(cromosoma):
    penalizaciones = 0

    sesiones_por_materia = defaultdict(lambda: defaultdict(int))  # materia -> día -> cantidad
    sesiones_por_hora = defaultdict(list)  # (día, hora) -> lista de materias

    materia_actual = None
    dia_actual = None
    hora_anterior = None

    # 1. Verificación de continuidad sin ordenar, aprovechando el orden del cromosoma
    for nivel, aula, dia, hora, materia, docente in cromosoma:
        dia = dia
        hora = int(hora)

        # Verificación de continuidad
        if materia != materia_actual or dia != dia_actual:
            # reiniciamos el seguimiento
            materia_actual = materia
            dia_actual = dia
            hora_anterior = hora
        else:
            # comparación directa, sin ordenar
            if horas_por_materia.get(materia, 0) > 1 and (hora - hora_anterior > 2):
                penalizaciones += 0.1

                # LOG de trazabilidad
                print(f"[SIN CONTINUIDAD] {materia} en {nivel}-{aula} con {docente} el día {dia} entre hora {hora_anterior} y {hora}")


            hora_anterior = hora

        # Recuento para otras restricciones
        sesiones_por_materia[materia][dia] += 1
        sesiones_por_hora[(dia, hora)].append(materia)

    # 2. Restricción: más de 3 horas continuas de una misma materia
    materia_anterior = None
    dia_anterior = None
    racha_continua = 1


    for i in range(1, len(cromosoma)):
        _, _, dia_act, hora_act, materia_act, _ = cromosoma[i]
        _, _, dia_ant, hora_ant, materia_ant, _ = cromosoma[i - 1]

        if nivel == "1EGB" and (materia == "MAT_1EGB" or materia == "LEN_1EGB"):
          continue

        if nivel == "2EGB" and materia == "MAT_2EGB":
          continue

        if materia_act == materia_ant and dia_act == dia_ant and int(hora_act) == int(hora_ant) + 1:
            racha_continua += 1
            if racha_continua > 3:
                penalizaciones += 2  # penalización moderada por exceso de continuidad
        else:
            racha_continua = 1  # reiniciar contador si cambia la materia o no es consecutiva

    # 3. Cruces de docentes en el mismo horario (con excepciones por nombre y cargo)
    docentes_en_tiempo = defaultdict(lambda: defaultdict(set))  # (día, hora) -> docente -> set de paralelos
    cargos_docentes = dict(zip(docentes["Nombres"], docentes["Cargo"]))  # asume que 'docentes' está cargado

    for nivel, paralelo, dia, hora, materia, docente in cromosoma:
        # Excepciones:
        if docente in {"EF ESPECIAL", "INGLES ESPECIAL"}:
            continue
        if docente == "JONATHAN CASTRO" and nivel == "3BGU":
            continue
        if docente == "JANETH MELO" and nivel == "3BGU":
            continue
        if materia == "CIU_1BGU" and  nivel=="1BGU":
            continue
        if materia == "CIU_2BGU" and  nivel=="2BGU":
            continue

        docentes_en_tiempo[(dia, hora)][docente].add(paralelo)

    for (dia, hora), docentes_dict in docentes_en_tiempo.items():
        for docente, paralelos in docentes_dict.items():
            if len(paralelos) > 1:
                # conflicto = f"[CRUCE] Docente: {docente} asignado a paralelos {sorted(paralelos)} el día {dia}, hora {hora}"
                # print(conflicto)
                penalizaciones += 5 * (len(paralelos) - 1)  # penalización fuerte por cruce real




    return penalizaciones

### ⚖️ 6. Evaluación de restricciones blandas (`evaluar_restricciones_blandas`)

Esta función penaliza configuraciones que afectan la calidad del horario, pero que no invalidan completamente la solución. Las penalizaciones son más suaves y permiten guiar al algoritmo evolutivo hacia horarios preferibles.

#### 📌 Subrestricciones consideradas:

1. **Docentes de medio tiempo:**
   - Si un docente con contrato `"Medio tiempo"` tiene clases asignadas a partir de la hora 11 (equivale a después de las 12pm), se penaliza con +2 por cada infracción.

2. **Docentes con licencia por maternidad:**
   - Si un docente con categoría `"Maternidad"` tiene clases después de la hora 14, se penaliza con +2 por cada infracción.

3. **(Reservado)**: `materias_por_dia` registra códigos por día y aula, con una lógica pensada para limitar la variedad de materias por día, pero esta parte aún no se ejecuta completamente (puedes extenderla para nuevas restricciones blandas en el futuro).

Este tipo de evaluación es útil para incorporar consideraciones humanas y operativas en el modelo, mejorando la aceptabilidad de los horarios generados.


In [ ]:
def evaluar_restricciones_blandas(cromosoma):
    penalizaciones = 0

    # Crear diccionarios rápidos de cargos y docentes
    cargo_docente = dict(zip(docentes["Nombres"], docentes["Cargo"]))

    sesiones_por_dia_docente = defaultdict(lambda: defaultdict(list))  # docente -> día -> horas
    materias_por_dia = defaultdict(set)

    for nivel, aula, dia, hora, materia, docente in cromosoma:
        if not materia.isupper():
            continue

        # 1. Horario de medio tiempo (mañana = hasta 12pm => Hora 8)
        if cargo_docente.get(docente) == "Medio tiempo" and int(hora) > 10:
            penalizaciones += 2

        # 2. Maternidad (última hora debe ser máximo la 14)
        if cargo_docente.get(docente) == "Maternidad" and int(hora) > 14:
            penalizaciones += 2


        # Registrar para posibles evaluaciones futuras
        sesiones_por_dia_docente[docente][dia].append(int(hora))

    for (aula, dia), codigos in materias_por_dia.items():
        if len(codigos) > 3:
            penalizaciones += (len(codigos) - 3)

    return penalizaciones

### 🧮 7. Función `fitness`: evaluación ponderada de calidad de un cromosoma

Esta función computa el valor de aptitud (`fitness`) de un cromosoma, combinando las penalizaciones de restricciones duras y blandas con ponderaciones ajustables.

#### ⚙️ Fórmula:
$
\text{fitness} = \frac{1}{1 + (\alpha \cdot \text{penalizaciones_duras} + \beta \cdot \text{penalizaciones_blandas})}
$

#### 📌 Parámetros:
- `cromosoma`: lista de genes con estructura `(nivel, aula, día, hora, materia, docente)`.
- `alpha`: peso de las restricciones duras (por defecto 1.0).
- `beta`: peso de las restricciones blandas (por defecto 0.2).

#### 📊 Resultado:
- El valor final estará en el rango \( (0, 1] \), donde valores cercanos a 1 indican horarios con muy pocas o ninguna violación de restricciones.
- Se penaliza fuertemente los errores duros y suavemente los errores blandos, ajustando el balance entre factibilidad y calidad del horario.

Esta métrica guía el proceso evolutivo, favoreciendo individuos que respetan las condiciones fundamentales del problema y, en segundo plano, optimizan su conveniencia operativa.


In [ ]:
# Función fitness con ponderaciones para restricciones duras y blandas
def fitness(cromosoma, alpha=1.0, beta=0.2):
    penal_duras = evaluar_restricciones_duras(cromosoma)
    penal_blandas = evaluar_restricciones_blandas(cromosoma)

    penal_total = alpha * penal_duras + beta * penal_blandas
    score = 1 / (1 + penal_total)

    return score

### 📚 8. Función `fitness_con_split`: evaluación global y por nivel

Esta función extiende el cálculo del fitness permitiendo obtener:
1. El fitness total del cromosoma.
2. El fitness individual por cada nivel educativo (EGB, BGU, etc.), útil para análisis por subgrupo o modularización de la evolución.

#### 📌 Descripción de la lógica:
- Se agrupan los genes del cromosoma por su campo `nivel`.
- Para cada grupo, se calcula el fitness utilizando la función estándar `fitness(...)`.
- También se calcula el fitness global del cromosoma completo.
- Se retorna una tupla con:
  - `fitness_total`: aptitud global considerando todo el cromosoma.
  - `fitness_por_split`: diccionario `{nivel: fitness_nivel}`.

#### 🎯 Aplicación:
Ideal para procesos de:
- **Trazabilidad**: detectar qué niveles están peor asignados.
- **División evolutiva**: evolucionar por niveles en paralelo o de forma secuencial.
- **Visualización por split**: generar gráficas de convergencia por nivel.


In [ ]:
def fitness_con_split(cromosoma, alpha=1.0, beta=0.2):
    from collections import defaultdict

    # Agrupar genes por nivel (gen[0] es el nivel)
    genes_por_nivel = defaultdict(list)
    for gen in cromosoma:
        nivel = gen[0]
        genes_por_nivel[nivel].append(gen)

    fitness_por_split = {}
    total_fitness = 0

    for nivel, subcromosoma in genes_por_nivel.items():
        # Calcular fitness del subcromosoma con función tradicional
        score_nivel = fitness(subcromosoma, alpha=alpha, beta=beta)
        fitness_por_split[nivel] = score_nivel
    total_fitness = fitness(cromosoma)

    return total_fitness, fitness_por_split

### 📂 9. Función `cargar_cromosoma_desde_txt`: lectura de cromosoma desde archivo `.txt`

Esta función permite cargar un cromosoma previamente guardado desde un archivo `.txt`, donde cada línea representa un gen en formato de tupla:

\[
(\text{nivel}, \text{aula}, \text{día}, \text{hora}, \text{materia}, \text{docente})
\]

#### 🧱 Lógica:
- Lee el archivo línea por línea.
- Ignora líneas vacías.
- Convierte cada línea en una tupla Python usando `eval()` y lo agrega a una lista.
- Retorna el cromosoma completo como una lista de genes.

#### 🛠️ Aplicaciones:
- Restaurar cromosomas elite (top 1, top 3) para análisis.
- Evaluar la calidad de soluciones almacenadas.
- Visualizar horarios previamente generados.


In [ ]:
def cargar_cromosoma_desde_txt(nombre_archivo):
    cromosoma = []
    with open(nombre_archivo, "r", encoding="utf-8") as f:
        for linea in f:
            linea = linea.strip()
            if linea:
                cromosoma.append(eval(linea))
    return cromosoma

### 🔍 10. Cargar e inspeccionar un cromosoma desde archivo

En este bloque se utiliza la función `cargar_cromosoma_desde_txt` para leer un cromosoma almacenado en un archivo `.txt` (por ejemplo, `"Horario_LEV.txt"`).

#### 🧾 Acciones realizadas:
- Se carga el cromosoma completo como lista de tuplas.
- Se imprime el total de genes (asignaciones).
- Se muestran los primeros genes para verificar su estructura y contenido.

Esto es útil para validar el archivo, inspeccionar asignaciones específicas, o alimentar procesos de evaluación, visualización o mutación evolutiva.


In [ ]:
cromosoma = cargar_cromosoma_desde_txt("Horario_LEV.txt")
print(f"Total de genes generados: {len(cromosoma)}")
# Mostrar los primeros 5 genes
for gen in cromosoma[:]:
    print(gen)

Total de genes generados: 2832
('1EGB', 'Ejemplares', 'Lunes', 1, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 2, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 3, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 4, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 6, 'DES_1EGB', 'ANDREA MACIAS')
('1EGB', 'Ejemplares', 'Lunes', 7, 'EDU_1EGB', 'EF ESPECIAL')
('1EGB', 'Ejemplares', 'Lunes', 8, 'EDU_1EGB', 'EF ESPECIAL')
('1EGB', 'Ejemplares', 'Lunes', 9, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 10, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 13, 'DAN_1EGB', 'PAOLA LÓPEZ')
('1EGB', 'Ejemplares', 'Lunes', 14, 'MUS_1EGB', 'LUIS JÁCOME')
('1EGB', 'Ejemplares', 'Martes', 1, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 2, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 3, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 4, 'EST_1EGB', 'SANTIAGO CALLE')
('1E

In [ ]:
fitness_con_split(cromosoma)

[SIN CONTINUIDAD] LEN_1EGB en 1EGB-Entusiastas con MALENA OÑA el día Martes entre hora 10 y 13
[SIN CONTINUIDAD] NAT_2EGB en 2EGB-Amistosos con MARIANA HIDALGO el día Miércoles entre hora 10 y 13
[SIN CONTINUIDAD] LEN_1EGB en 1EGB-Entusiastas con MALENA OÑA el día Martes entre hora 10 y 13
[SIN CONTINUIDAD] NAT_2EGB en 2EGB-Amistosos con MARIANA HIDALGO el día Miércoles entre hora 10 y 13


(0.16129032258064516,
 {'1EGB': 0.9090909090909091,
  '2EGB': 0.9090909090909091,
  '3EGB': 1.0,
  '4EGB': 1.0,
  '5EGB': 1.0,
  '6EGB': 1.0,
  '7EGB': 1.0,
  '8EGB': 1.0,
  '9EGB': 1.0,
  '10EGB': 1.0,
  '1BGU': 1.0,
  '2BGU': 1.0,
  '3BGU': 1.0})